In [1]:
#Title 

In [2]:
#Deliverable 1 Population Statistics

In [3]:
!pip install wbdata

  Using cached wbdata-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached dateparser-1.2.0-py2.py3-none-any.whl.metadata (28 kB)
  Using cached shelved_cache-0.3.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached tabulate-0.8.10-py3-none-any.whl.metadata (25 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
Using cached wbdata-1.0.0-py3-none-any.whl (18 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached dateparser-1.2.0-py2.py3-none-any.whl (294 kB)
Using cached shelved_cache-0.3.1-py3-none-any.whl (7.8 kB)
Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
Using cached tzlocal-5.2-py3-none-any.whl (17 kB)


In [4]:
import wbdata
wbdata.get_topics()

Key '8412339339633258484' not in persistent cache.
Key '4136494290281043500' not in persistent cache.
Key '-98531610836527358' not in persistent cache.
Key '7564657874146399076' not in persistent cache.
Key '-2983001891162961056' not in persistent cache.
Key '-7363016685015268271' not in persistent cache.
Key '2922340212054388173' not in persistent cache.
Key '1957624295938988752' not in persistent cache.
Key '-6893486418599443809' not in persistent cache.
Key '5476693810486873021' not in persistent cache.
Key '4902340800987033039' not in persistent cache.
Key '2034164340195906102' not in persistent cache.
Key '7231440316382657029' not in persistent cache.
Key '1082137934057665025' not in persistent cache.
Key '-2608153321896104801' not in persistent cache.
Key '4601207448605549294' not in persistent cache.
Key '234456468570805617' not in persistent cache.
Key '-9159147850551378477' not in persistent cache.
Key '-8995816368684581207' not in persistent cache.
Key '5175082655672755425' n

  id  value
----  -------------------------------
   1  Agriculture & Rural Development
   2  Aid Effectiveness
   3  Economy & Growth
   4  Education
   5  Energy & Mining
   6  Environment
   7  Financial Sector
   8  Health
   9  Infrastructure
  10  Social Protection & Labor
  11  Poverty
  12  Private Sector
  13  Public Sector
  14  Science & Technology
  15  Social Development
  16  Urban Development
  17  Gender
  18  Millenium development goals
  19  Climate Change
  20  External Debt
  21  Trade

In [5]:
import pandas as pd
import numpy as np

In [6]:
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Glob

In [7]:
indicators = wbdata.get_indicators(source=40)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

In [8]:
sexdict = {'Male':'MA', 'Female':'FE'}
def population(year,sex,age_range,place):
    lower, upper = age_range[0], age_range[1]
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df = df[df['date'] == f'{year}-01-01'] # selecting relevant year
        df.drop('date', axis=1, inplace=True)
        extra = ((yrs[-1][1] - upper) / 5) * (df.iloc[:,-1].to_list()[0]) # controls for age ranges not on bounds, could change to 
                                                                          # year by year subtraction)
    return df.sum(axis=1) - extra

In [9]:
population(1990, 'Male', (10,33), 'CHN')

33    263801536.6
dtype: float64

In [27]:
def population_data_frames(year, sex, age_range, place):
    lower, upper = age_range[0], age_range[1]
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df['country'] = place
        df.set_index(['country', 'date'], inplace=True)
    return df

In [28]:
population_data_frames(1990, 'Male', (10,33), 'CHN')

Male 1014   Male 1519   Male 2024   Male 2529   Male 3034
country date                                                                  
CHN     2023-01-01  47697475.0  43619845.0  42690196.0  45996968.0  58571312.0
        2022-01-01  46945694.0  42880142.0  43079433.0  47322938.0  61347262.0
        2021-01-01  46077575.0  42441714.0  43637348.0  48943931.0  63431979.0
        2020-01-01  45390757.0  42444894.0  44208685.0  51917714.0  63016755.0
        2019-01-01  44534610.0  42626158.0  45014490.0  55706672.0  60783908.0
...                        ...         ...         ...         ...         ...
        1964-01-01  45210547.0  32772310.0  27179605.0  26566937.0  24330081.0
        1963-01-01  43265071.0  31245247.0  27269359.0  26878889.0  23871048.0
        1962-01-01  40877563.0  29902774.0  27482213.0  26824171.0  23384228.0
        1961-01-01  38596810.0  29125584.0  27700640.0  26562875.0  23028989.0
        1960-01-01  36823111.0  29027168.0  28092415.0  26459425.0  22874774.0

[64 rows x 5 columns]